In [1]:
#shell of notebook

#Company Report Imports
import pandas as pd

import plotly.graph_objects as go

import networkx as nx
import ast
%matplotlib inline  
#from networkx.drawing.nx_agraph import write_dot, graphviz_layout
import matplotlib.pyplot as plt

#Company Suggestions Imports
import re
import sys
from IPython.display import display, HTML

#Display Dropdown Imports
import ipywidgets as widgets
from IPython.display import display
import functools

#Display Button Imports:
import asyncio



# 0. Core/Background Functions

### Display Dropdown

In [2]:
def display_drop_down(df_column):
    """
    Passes in a Dataframe column to set the dropdown bar. Use in tandem with button code
    to create clickable button to direct users to different functions
    """
    dd = widgets.Dropdown(options= 
          df_column.unique(),
    value=df_column.iloc[0],
    description='Type:',
    disabled=False,
    )

    return dd

dd = display_drop_down(sic_sectors_codes['sector_group_name']) #set this to the column you like to define the code 


NameError: name 'sic_sectors_codes' is not defined

In [3]:
def unique_non_nan(elem):
    """
    To be used in aggregation functions, rolls elements into a list, discaring nans
    """
    return [a for a in list(pd.unique(elem)) if pd.notna(a)]

def deduplicate(col, ls):
    if col in ls: ls.remove(col)
    return ls

In [4]:
#Column Names 
PARENT_CW_ID = "parent_cw_id"
CHILD_COMPANY_NAMES = "child_company_names"
CHILD_CW_IDS = "child_cw_ids"
PARENT_CW_IDS = "parent_cw_ids"
PARENT_COMPANY_NAMES = "parent_company_names"
CW_ID = 'cw_id'
INDUSTRY_NAME = 'industry_name'
SECTOR_NAME = 'sector_name'
NUM_PARENTS = 'num_parents'
NUM_CHILDREN = 'num_children'
TOP_PARENT_ID = 'top_parent_id'
COMPANY_NAME = 'company_name'
CHILD_CW_ID_LIST = 'child_cw_id_list'
PARENT_CW_ID_LIST = 'parent_cw_id_list'


In [19]:
def create_master_company_data(output_name, corp_watch_dir = "corpwatch"):
    company_info = pd.read_csv('corpwatch_api_tables_csv/company_info.csv', sep = "\t", escapechar='\\',  low_memory=False)
    recent_companies = company_info[company_info['most_recent'] == 1]
    
    #read relationships file which we will use to show parent and child relationships 
    relationships = pd.read_csv('corpwatch_api_tables_csv/relationships.csv', sep = "\t", escapechar='\\')
    relationships_recent = relationships[relationships['year'] > 2018]
    
    #group by the parent cw id, to get a list of children 
    parent_child_groups = relationships_recent.groupby(['parent_cw_id']).agg({
     'parent_cw_id': "first",
     'cw_id': unique_non_nan,
     'clean_company': unique_non_nan})
    
    parent_child_groups['child_cw_id_list'] = parent_child_groups.apply(lambda row: deduplicate(row['parent_cw_id'], row.cw_id), axis = 1)
    #parent_child_groups.rename(index = {'parent_cw_id': "cw_id"}, inplace = False)
    parent_child_groups = parent_child_groups.rename(
    columns= {
    "cw_id": CHILD_CW_IDS,
    'clean_company': CHILD_COMPANY_NAMES})

    parent_child_groups = parent_child_groups[['child_cw_id_list']]
    
    #group by the cw_id to get a list of parents for each cw id 
    child_parent_groups = relationships_recent.groupby(['cw_id']).agg({
     'parent_cw_id' : lambda x:  unique_non_nan(x),
     'clean_company': lambda x: unique_non_nan(x)})
    child_parent_groups['child_cw_id'] = child_parent_groups.index
    child_parent_groups['parent_cw_id_list'] = child_parent_groups.apply(lambda row: deduplicate(row['child_cw_id'], row['parent_cw_id']), axis =1)
   
    company_joined1 = pd.merge(recent_companies,
                     parent_child_groups, 
                     left_on= 'cw_id',
                     right_on='parent_cw_id',          
                     how='left')
    
    company_joined2 =  pd.merge(company_joined1,
                     child_parent_groups, 
                     left_on= 'cw_id',
                     right_on='child_cw_id',
                     how='left')
    company_joined2.to_csv(output_name)

In [20]:
#Run the code to create the company master table 
COMPANY_MASTER_TABLE_NAME = "company_master_table.csv"
#Only need to run this code once 
#create_master_company_data(COMPANY_MASTER_TABLE_NAME)

In [24]:
#required to read the list column from a file 
def list_parse(s):
    strings = str(s).strip("[]").split(",")
    ls = []
    for i in strings:
        try:
            ls.append(int(float(i.strip())))
        except (ValueError):
            ""
    return ls

## Code to read in the company file

In [25]:
#read in company master file, parse, add location information 

def read_company_file():
    data = pd.read_csv("company_master_table.csv", 
                   dtype={"child_cw_id_list" :'str',
                          "parent_cw_id_list": 'str'}, converters={1:ast.literal_eval})
    return data

def clean_company_data(data):
    data[CHILD_CW_ID_LIST] = data[CHILD_CW_ID_LIST].apply(lambda x: list_parse(x))
    data[PARENT_CW_ID_LIST] = data[PARENT_CW_ID_LIST].apply(lambda x: list_parse(x))
    families = data[((data[NUM_PARENTS]> 0) | (data[NUM_CHILDREN] > 0))]  #i might not want to do this
    
    #join with the location 
    location = pd.read_csv('corpwatch_api_tables_csv/company_locations.csv', sep = "\t", escapechar='\\')
    location_filtered = location[location['max_year'] > 2018]
    location_aggregations = {'city': "first",
                         'state': "first",
                         'country_code':  "first"}
    location_agg = location_filtered.groupby('cw_id').agg(location_aggregations)
    
    families_location = pd.merge(families, location_agg,
                     left_on='cw_id', 
                     right_on='cw_id',
                     how='left')
    #index on cw_id for faster lookup 
    indexed = families_location.set_index(CW_ID) ## should not use the filtered list 
    return indexed

In [26]:
company_master_data = read_company_file()
company_data = clean_company_data(company_master_data)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning:

Columns (5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.



,Unnamed: 0,row_id,most_recent,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,...,min_year,no_sic,child_cw_id_list,parent_cw_id,clean_company,child_cw_id,parent_cw_id_list,city,state,country_code
cw_id,,,,,,,,,,,,,,,,,,,,,
37,0,18,1,2019,3153,630004250.0,74,4911.0,Electric services,4900.0,...,2003,0,"[1665, 993179, 101861, 1018259, 781, 101870, 1...","[12009.0, 22569.0, 781.0, 1197.0, 1665.0]",['Alabama Power Co'],37.0,"[12009, 22569, 781, 1197, 1665]",BIRMINGHAM,AL,US
45,1,24,1,2019,3499,510100517.0,88,6798.0,Real estate investment trusts,6700.0,...,2003,0,"[1038175, 1038176, 1038177, 1038178, 1038179, ...","[14480.0, 9019.0]",['Alexanders Inc'],45.0,"[14480, 9019]",PARAMUS,NJ,US
69,2,39,1,2019,4515,131502798.0,137,4512.0,"Air transportation, scheduled",4500.0,...,2003,0,"[964843, 964842, 411832, 411833, 416126, 41613...",[105.0],['American Airlines Inc'],69.0,[105],FT WORTH,TX,US
120,3,65,1,2019,6879,540124790.0,246,4911.0,Electric services,4900.0,...,2003,0,"[993634, 834826, 696650, 992937, 136825, 95658...","[956587.0, 1029138.0, 75.0, 927.0, 1345.0, 149...",['Appalachian Power Co'],120.0,"[956587, 1029138, 75, 927, 1345, 1492, 1674]",COLUMBUS,OH,US
130,4,79,1,2019,7323,831918668.0,273,4911.0,Electric services,4900.0,...,2003,0,"[899487, 1001712, 1180]",[590084.0],['Entergy Arkansas LLC'],130.0,[590084],LITTLE ROCK,AR,US


### Button Code

In [ ]:
slider = IntSlider()

async def f():
    for i in range(10):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider

### Weighted Results

In [ ]:
#### weighting for search results

def weighted_result(x, sector):
    #use zone scoring to compute weight of query result    
    child_weight = .5
    sector_weight = .3
    company_weight = .2
    
    #returns positive if there are any children (favors multinational companies)
    child = (0,1)[x['num_children'] > 0]
    #returns positive if the company is an INC or CORP datatype (More likely to be a US multinational)
    company = (0,1)[x['company_type'] == 'INC' or x['company_type'] == 'CORP' or x['company_type'] == NaN]
    
    #attempts to put sector(if given) into related companies (replace with dropdown if possible)
    try:
        sect = (0,1)[sector.lower() in x['sector_name'].lower()]
    except:
        sect = 0

    return child_weight * child + sector_weight * sect + company_weight * company


## List Parse

# A. Start Page

Welcome to the Multinational 

### Workflow
1. Users choose what kind of search they want


### Choices

- Choose a Company (Go to -> __C. Company Query__)

- Choose a Sector (Go to -> __B. Search by Sector__)

- (Optional) Choose from top multinational companies


##### Disclaimer
Only includes US companies

In [ ]:
def AStart():
    '''Display start page + options'''
    #B = Sector
    #C = Company
    while True:
        choice = input("Please select an option: ")
        if choice == "B":
            BSector()
        elif choice == "C":
            CCompany()
        else:
            print("sorry I didn't get that, please try again")

## B. Search by Sector

### Workflow
1. Users select Sector Group (dropdown)
1b. Users select Sector (dropdown)
1c. Users select Industry (dropdown)

2. Top Players Results displayed
3. Users select company to view report (dropdown) (Go to -> __D. Company Report Page__)
3b. Users go back (Go to -> __A. Start Page__)

### Features
- Dropdown Widgets: Sector selection
- Top Players Results
    - Groupby: Industries
    - Groupby: Top Parent IDs
- Dropdown Widgets: Company selection



In [ ]:
def BSector():
    '''display sectors options'''
    display_drop_down(Sector_Groups)
    display_drop_down(Sectors)
    display_drop_down(Industries)
    SIC_choice = "Sector/industry"
    
    search_by_sector(SIC_choice)
    #displays top 25 results
    
    company_choice = display_drop_down(Companies)
    display_company_report = company_choice

    

In [28]:
# rachel will start
def display_drop_down(options_list):
    """
    displays a drop down for the items in options list, collects choice from the user and returns a string with
    their choice
    
    Requirements: Sector_group_name, Sector_name, Industry_name
    """
    choice = "Berkshire Hathway"
    print(options_list)
    choice = input('Type the item you would like to choose')
    return choice

In [ ]:
#prakar
def search_by_sector(sector_name):
    """
    returns list of a parent companies in that sub sector, ranked by subsidiaries
    """
    return []

## C. Search by Company

### Workflow
1. Users search by Company Name
1b. Users select Sector (dropdown)
1c. Users select Industry (dropdown)

2. Company Results displayed
3. Users select company to view report (dropdown) (Go to -> __D. Company Report Page__)
3b. Users go back (Go to -> __A. Start Page__)

### Features
- Dropdown Widgets: Sector selection
- Weighted Zone Search
    - Number of Children
    - Company Type
    - Sector Type
- Dropdown Widgets: Company selection

In [ ]:
def CCompany():
    company_suggestions()
    

In [ ]:
# Max is going to write this to start with
#company_joined2['weighted_result'] = 'default'
import re
import sys
from IPython.display import display, HTML
def company_suggestions(company_joined2):
    '''
    if name isn't clean, then search name+wildcard to return list of companies, ranked by number of children.
    can remove
    '''
    
    '''
    first test if name entered finds something in the database
    '''
    
    print('Please input a company you would like to search for')
    print('Tips for searching:')
    print('-You can enter a partial number')
    company = input('> ').upper()
    
    sector = input('What sector are you looking for: ')
    #company_joined2.columns
    df = pd.DataFrame(company_joined2, columns = ['cw_id', 'company_name', 'num_children', 'sector_name', 'weighted_result', 'company_type'])
#     df.head()
    #display_company_report(company) #navigates to company report if selected
    df = df[ df['company_name'].str.match(company) ]
    if df.size == 0:
        print("I'm sorry, I wasn't able to find a company with those attributes. Would you like to do next:")
        print("1) Search for another company?")
        print("2) Go Back")
        company = input('> ').upper()

    df['weighted_result'] = df[['num_children','sector_name','company_type']].apply(lambda x: weighted_result(x,sector), axis=1)
    df = df.sort_values(by=['weighted_result'], ascending = False)
    display(df)
    #RACHEL"S NOTE: This function should end here and return a tuple with the dataframe and a list
    #The code below should be in the top level of the notebook 
    x = 0
    while x < 3:
        choice = None
        if df.size == 0:
            print("I'm sorry, I wasn't able to find a company with those attributes. Would you like to do next:")
            print("1) Search for another company?")
            print("2) Go Back")
            choice = int(input("> "))
        elif df.size != 1:
            print('which company would you like to select (cw_id):')   
            cw = int(input('>'))
            company = df.loc[df['cw_id'] == cw, ['company_name', 'child_cw_id_list'] ]
        else:
            company = df.iloc[0]
            print(company)

        if choice != None:
            print('What would you like to do now:')
            print('1) Search for another Company')
            print('2) Go Back')
            print('3) Get Company Report')
            choice = int(input("> "))
        else:
            if choice == 1:
                company_suggestions()
            elif choice == 2:
                test = ''
                #go back to previous page
            elif choice == 3:
                display_breakdown(company, cw)
            else:
                print('Sorry I didnt get that, please select another choice')
        x+=1
    print('Sorry you have failed too many times. Please try again')
    sys.exit()
    
    return None#ranked list of company suggestions
#company_suggestions()

## D. Company Report Page

### Workflow

1. User is shown page
2. User select another company to view report (dropdown) (Go to -> __D. Company Report Page__)
2b. Users go back (Go to -> __A. Start Page__)


### Features

#### Parent Company
- Relationship Map
- Company Breakdown (% of Company Industries)
    - Groupby: Top Parent ID
    - Groupby: Sector
    - Top Players Ranking
        - Top Players Results
- Foreign/Domestic (% of Domestic vs. International Companies)
    - Groupby: Location
    - Groupby: State
- Dropdown Widgets: Company selection


#### Subsidiary
- Similar Companies
    - Groupby: Top Parent ID
    - Groupby: Sector
- Dropdown Widgets: Company selection

In [ ]:
#rachel with start
def display_company_report(clean_company_name):
    """
    given a clean company name, show a display report of the company 
    
    SIC 
    LOC
    F/D % percent of companies that are abroad vs. domestic
    
    
    Requirements: Corpwatch Company data, company relationship data

    """
    display_breakdown(clean_company_name)
    display_hierarchy(clean_company_name)
    
    company = input('')
    display_company_report(company)
    #display something
    #return the report

In [ ]:
def display_breakdown(clean_company_name):
    '''
    uses logic to determine if it should display (e.g. is it a parent)
    
    if so, shows Pie chart of sector breakdown
    
    shows list of sectors they are in
    '''
    return []

In [ ]:
def display_hierarchy(clean_company_name):
    '''
    uses logic to determine if it should display (e.g. does it have at least a parent or a child)
    
    if so, show table of parent (if applicable)
    show table of children (if applicable)
    '''
    return []

In [ ]:
if result == 'sector':
    #show sector group
    
    sector_company_list = search_by_sector(sector_name): #returns all companies in sector
    company = display_dropdown(sector_company_list) #displays companies with subsidiaries 
    display_company_report(company) #navigates to company report if selected
    #show sector name
    #show industry name
    

# ACTUAL PAGE

In [ ]:
print("Enter the name of a ")